# Model (ResNet) - Binary

Within this model, we'll use the same VGG16 model, but reduce the options down to 'Subuaru' and 'Not Subaru'

### Library Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import pickle
import time
import os
import json
import seaborn as sns
import tensorflow as tf
import random
import cv2
import copy

import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import torchvision.transforms as transforms



from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.models import Sequential

from keras.applications import VGG16
from keras.applications.resnet import ResNet152
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from datetime import datetime

from pandas import json_normalize
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay,accuracy_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, precision_score, recall_score, f1_score,classification_report
from sklearn.preprocessing import normalize,StandardScaler,RobustScaler,MinMaxScaler
from sklearn.svm import SVC,LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.compose import ColumnTransformer



from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Flatten, Conv2D, MaxPooling2D,LeakyReLU
from tensorflow.keras.regularizers import l1, l2, l1_l2
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from keras.callbacks import EarlyStopping
from tensorflow.keras.datasets import mnist, cifar10

from tensorflow import keras
from keras.utils import to_categorical
from keras import regularizers
from keras.models import load_model
from keras.layers import GlobalAveragePooling2D
from keras.losses import binary_crossentropy

from scipy.stats import skew
from tqdm import tqdm
from PIL import Image

### Data Augmentation

In [ ]:
# Remove comment to run again
# datagen = ImageDataGenerator(
#     rotation_range=15,
#     width_shift_range=0.15,
#     height_shift_range=0.15,
#     shear_range=0.1,
#     zoom_range=0.4,
#     horizontal_flip=True,
#     vertical_flip=False,
#     fill_mode='nearest'
# )

In [ ]:
# image_dir = "Datasets/BoundingBoxes/"

# for root, dirs, files in os.walk(image_dir):
#     for filename in files:
#         if not filename.startswith('.'):
#             if os.path.basename(root).startswith('subaru'):
#                 filepath = os.path.join(root, filename)
#                 # load the image and convert it to a numpy array
#                 img = load_img(filepath)
#                 img_array = img_to_array(img)
#                 # reshape the array to have an extra dimension
#                 img_array = img_array.reshape((1,) + img_array.shape)
#                 # generate X (in range) augmented images and save them
#                 for i in range(4):
#                     aug_img_array = datagen.flow(img_array, batch_size=1)[0]
#                     aug_img = aug_img_array.astype('uint8')
#                     new_filename = f"{filename.split('.')[0]}_aug{i}.jpg"
#                     new_filepath = os.path.join(root, new_filename)
#                     # save the augmented image
#                     Image.fromarray(aug_img.squeeze()).save(new_filepath)
#                     print(f"New file created: {new_filepath}")

### Generator (Data Generator)

In [ ]:
image_dir = "Datasets/BoundingBoxes/"

In [ ]:
normalize = transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])

def image_data_generator(image_dir, batch_size):
    images = []
    labels = []
    for brand_dir in os.listdir(image_dir):
        if not brand_dir.startswith('.'):
            if 'subaru' in brand_dir.lower():
                label = 1
            else:
                label = 0
            for filename in os.listdir(os.path.join(image_dir, brand_dir)):
                if not filename.startswith('.'):
                    filepath = os.path.join(image_dir, brand_dir, filename)
                    image = cv2.imread(filepath)
                    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                    image = cv2.resize(image, (224, 224))
                    images.append(image)
                    labels.append(label)
                    if len(images) == batch_size:
                        images = np.array(images)
                        images = images.transpose((0, 3, 1, 2))  # swap color axis
                        images = torch.from_numpy(images).float()
                        images /= 255.0
                        images = normalize(images)
                        labels = torch.from_numpy(np.array(labels)).long()
                        yield images, labels
                        images = []
                        labels = []
    if len(images) > 0:
        images = np.array(images)
        images = images.transpose((0, 3, 1, 2))  # swap color axis
        images = torch.from_numpy(images).float()
        images /= 255.0
        images = normalize(images)
        labels = torch.from_numpy(np.array(labels)).long()
        yield images, labels

In [ ]:
generator = image_data_generator(image_dir, batch_size=32)

In [ ]:
all_images = []
all_labels = []
for batch_images, batch_labels in generator:
    all_images.append(batch_images)
    all_labels.append(batch_labels)

# Concatenate the data
all_images = np.concatenate(all_images, axis=0)
all_labels = np.concatenate(all_labels, axis=0)
all_labels = np.array([str(label).split("_")[0] if isinstance(label, str) else label for label in all_labels])

In [ ]:
num_classes = len(np.unique(all_labels))
num_classes

### Randomly Sampling

In [ ]:
non_subaru_mask = all_labels == 0

In [ ]:
sample_size = int(1 * np.sum(non_subaru_mask)) ### Currently set to 1 to include all data
sample_indices = np.random.choice(np.where(non_subaru_mask)[0], size=sample_size, replace=False)

In [ ]:
subsampled_images = all_images[np.concatenate([sample_indices, np.where(all_labels == 1)[0]])]
subsampled_labels = all_labels[np.concatenate([sample_indices, np.where(all_labels == 1)[0]])]

### Train Test Split

To Train-Test split the data, we'll be seperating X on index to prevent all of the data from being loaded into the kernel concurrently.

In [ ]:
idx = np.arange(subsampled_images.shape[0])
train_idx, test_idx = train_test_split(idx, test_size=0.2, stratify=subsampled_labels)

In [ ]:
X_train, y_train = subsampled_images[train_idx], all_labels[train_idx]

In [ ]:
X_test, y_test = subsampled_images[test_idx], subsampled_labels[test_idx]

In [ ]:
# y_train = np.reshape(y_train, (-1, 1))
# y_test = np.reshape(y_test, (-1, 1))

In [ ]:
len(y_test)

In [ ]:
print('Subaru count:', np.sum(y_train))
print('Other count:', np.sum(1 - y_train))

### Data Generators

We'll feed our data into the model in batches of 32 so that all images aren’t loaded into the kernel at the same time. Within this function, we'll divide X_low by 255, to normalize the data.

This operation is split into two seperate generators to prevent data leakage from the training set into the validation set. Since the function has been seperated, each function will only be called when needed.

In [ ]:
def train_data_generator(X_train, y_train, batch_size):
    num_train_samples = len(X_train)
    train_indices = np.arange(num_train_samples)
    np.random.shuffle(train_indices)

    while True:
        for start_idx in range(0, num_train_samples, batch_size):
            end_idx = min(start_idx + batch_size, num_train_samples)
            batch_indices = train_indices[start_idx:end_idx]
            X_batch_train = torch.tensor(X_train[batch_indices]).float()
            y_batch_train = torch.tensor(y_train[batch_indices]).long()

            # Normalize the input data to [0, 1]
            X_batch_train /= 255.0

            yield (X_batch_train, y_batch_train)

In [ ]:
def test_data_generator(X_test, y_test, batch_size):
    num_test_samples = len(X_test)
    test_indices = torch.randperm(num_test_samples)
    while True:
        for start_idx_test in range(0, num_test_samples, batch_size):
            end_idx_test = min(start_idx_test + batch_size, num_test_samples)
            batch_indices_test = test_indices[start_idx_test:end_idx_test]
            batch_X_test = X_test[batch_indices_test]
            batch_y_test = y_test[batch_indices_test]
            yield batch_X_test, batch_y_test

In [ ]:
batch_size = 32
train_generator = train_data_generator(X_train, y_train, batch_size)
val_generator = test_data_generator(X_test, y_test, batch_size)

In [ ]:
train_steps = len(X_train) // batch_size
val_steps = len(X_test) // batch_size

### Instantiating VGG16 Model (TensorFlow)

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.00001)
loss_fn = tf.keras.losses.BinaryCrossentropy()
metrics=['accuracy']
epochs = 50

In [ ]:
base_model = ResNet152(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

for layer in base_model.layers:
    layer.trainable = False

In [ ]:
model = Sequential([
    base_model,
    Flatten(),
    Dense(1, activation='sigmoid')
])

In [ ]:
model.compile(optimizer= optimizer,
              loss= loss_fn,
              metrics= metrics)

In [ ]:
early_stopping = EarlyStopping(monitor='val_accuracy', patience=10)


In [ ]:
model.fit(train_generator,
          epochs=50,
          validation_data=val_generator,
          callbacks=[early_stopping],
          steps_per_epoch=train_steps, 
          validation_steps=val_steps)

### PyTorch Model

* To do:
 - seperate train/test datasets
 - create pytorch dataloaders for each datasets like
 

In [ ]:
# Put them in a dictionary
dataloaders = {'train': train_data_generator, 'valid': test_data_generator}

In [ ]:
if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    x = torch.ones(1, device=mps_device)
    print (x)
else:
    print ("MPS device not found.")

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3)
        self.conv2 = nn.Conv2d(32, 64, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 53 * 53, 32)
        self.fc2 = nn.Linear(32, 2)
        self.dropout = nn.Dropout2d(p=0.2)
    
    def forward(self, x):
        x = F.leaky_relu(self.conv1(x))
        x = self.pool(F.leaky_relu(self.conv2(x)))
        x = x.view(-1, 64 * 53 * 53)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return F.softmax(x, dim=1)

In [ ]:
def train_model(model, criterion, optimizer, num_epochs=5, device='cuda'):
    start = time.time()
    train_results = []
    valid_results = []
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10)
        # Each epoch has a training and validation phase
        for phase in ['train', 'valid']:
            if phase == 'train':            
              model.train()  # Set model to training mode
            else:
              model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward pass
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            
            if(phase == 'train'):
              train_results.append([epoch_loss,epoch_acc])
            if(phase == 'valid'):
              valid_results.append([epoch_loss,epoch_acc])
                                   
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model (Early Stopping) and Saving our model, when we get best accuracy
            if phase == 'valid' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())       
                model_save_name = "resnetCars.pt"
                path = F"/content/drive/My Drive/{model_save_name}"
                torch.save(model.state_dict(), path)        

        print()

    # Calculating time it took for model to train    
    time_elapsed = time.time() - start
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    #load best model weights
    model.load_state_dict(best_model_wts)
    
    return model, train_results, valid_results

In [ ]:
model = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

model, train_results, valid_results = train_model(model, criterion, optimizer, num_epochs=5)

### Gradient Tape

In [ ]:
y_train = np.hstack((y_train, 1 - y_train))
y_test = np.hstack((y_test, 1 - y_test))

In [ ]:
def train(model, optimizer, loss_fn, train_data_generator, X_train, y_train, batch_size, epochs):
    for epoch in range(epochs):
        epoch_loss = tf.Variable(0.0, dtype=tf.float32)
        epoch_accuracy = tf.Variable(0.0, dtype=tf.float32)
        batches = train_data_generator(X_train, y_train, batch_size)
        with tqdm(total=len(X_train)//batch_size) as pbar:
            for i, (X_batch_train, y_batch_train) in enumerate(batches):
                with tf.GradientTape() as tape:
                    y_pred = model(X_batch_train, training=True)
                    batch_loss = loss_fn(y_batch_train, y_pred)
                gradients = tape.gradient(batch_loss, model.trainable_variables)
                optimizer.apply_gradients(zip(gradients, model.trainable_variables))
                batch_accuracy = accuracy_score(np.argmax(y_batch_train, axis=1), np.argmax(y_pred.numpy(), axis=1))
                epoch_loss.assign_add(tf.reduce_sum(batch_loss))
                epoch_accuracy.assign_add(batch_accuracy)
                pbar.update(1)
        epoch_loss = epoch_loss / (X_train.shape[0] // batch_size)
        epoch_accuracy = epoch_accuracy / (X_train.shape[0] // batch_size)
        print(f"Epoch {epoch + 1}: loss={epoch_loss}, accuracy={epoch_accuracy}")

In [ ]:
def test(model, loss_fn, test_data_generator, X_test, y_test, batch_size):

    # Initialize the loss and accuracy
    test_loss = 0.0
    test_accuracy = 0.0

    # Iterate over the batches in the test data generator
    for X_batch_test, y_batch_test in test_data_generator(X_test, y_test, batch_size):

        # Compute the predictions and loss for the batch
        y_pred = model(X_batch_test, training=False)
        batch_loss = loss_fn(y_batch_test, y_pred)

        # Compute the batch accuracy
        batch_accuracy = accuracy_score(np.argmax(y_batch_train.numpy(), axis=1), np.argmax(y_pred.numpy(), axis=1))

        # Update the test loss and accuracy
        test_loss += batch_loss.numpy()
        test_accuracy += batch_accuracy

    # Compute the average test loss and accuracy
    test_loss /= (len(X_test) / batch_size)
    test_accuracy /= (len(X_test) / batch_size)

    # Print the test loss and accuracy
    print("Test loss: {:.4f} - Test accuracy: {:.4f}".format(test_loss, test_accuracy))

    return test_loss, test_accuracy

In [ ]:
train(model, optimizer, binary_crossentropy, train_data_generator, X_train, y_train, batch_size, epochs)

In [ ]:
tf.keras.backend.clear_session()
early_stopping = EarlyStopping(monitor='val_loss', patience=3)
log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
history = model.fit(
    train_data_generator(X_train, y_train, batch_size),
    validation_data=test_data_generator(X_test, y_test, batch_size),
    epochs=epochs, 
    steps_per_epoch=train_steps, 
    validation_steps=val_steps,
    callbacks=[early_stopping, tensorboard_callback]
)

In [ ]:
test_loss, test_acc = test(model, loss_fn, test_data_generator, X_test, y_test, batch_size)

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
y_pred = model.predict(X_test)
y_pred_labels = np.round(y_pred)
cm = confusion_matrix(y_test, y_pred_labels)

In [ ]:
cm